In [1]:
# Library imports.
from pyswmm import Simulation, Links, Output
from swmm.toolkit.shared_enum import LinkAttribute, NodeAttribute, SubcatchAttribute
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import numpy.random as nprnd
from scipy.interpolate import griddata
from sklearn.linear_model import LinearRegression
import time

# Local imports.
from urbansurge import swmm_model, analysis_tools, file_utils

%load_ext autoreload
%autoreload 2

In [2]:
value_type = 'fraction'

# Fault components.
# Weirs.
fault_type = 'diameter'
fault_components = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 30, 31, 32, 35, 36, 64]
fault_values = [0.1, 0.3, 0.6, 0.9]

# Links.
fault_type = 'roughness'
fault_components = [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65]
fault_values = [2, 3, 4, 5, 6]

# Fault component, fault values.
fault_value_bulk = {i:fault_values for i in fault_components}

### Impulse at single location.

In [35]:
# Input node where impulse was added.
input_node = 18

# Create instance of SWMM model.
config_path = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_config.yml"
swmm = swmm_model.SWMM(config_path)

# Configure model.
swmm.configure_model()

# All downstream links from input node.
downstream_links = swmm.get_downstream_components(input_node, 'Link')
print(downstream_links)

#### Diameter fault.
# Find all links with the "clog" tag downstream of input node. Diameter fault.
clog_links = swmm.get_components_by_tag('clog')
clog_links = [int(l) for l in clog_links]
fault_components = np.intersect1d(downstream_links, clog_links)
print('DIAMETER')
print(fault_components)
print(len(fault_components), '\n')
fault_type = 'diameter'
fault_values = np.linspace(0.1, 0.9, 9)

#### Roughness fault.
# Find all links downstream of input node that don't have "clog" tag. Roughness fault.
fault_components = [l for l in downstream_links if l not in clog_links]
print('ROUGHNESS')
print(fault_components)
print(len(fault_components), '\n')
fault_type = 'roughness'
fault_values = np.linspace(1.5, 6, 10)

#### Storage fault.
fault_components = [20]
fault_type = 'silting'
fault_values = np.linspace(0.1, 0.9, 21)
print('SILTING')
print(fault_components)
print(len(fault_components), '\n')

# Fault component, fault values.
fault_value_bulk = {i:fault_values for i in fault_components}
print(fault_value_bulk)

Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
[39, 77, 40, 41, 78, 42, 43, 79, 44, 45, 80, 23, 21, 76, 20]
DIAMETER
[76 77 78 79 80]
5 

ROUGHNESS
[39, 40, 41, 42, 43, 44, 45, 23, 21, 20]
10 

SILTING
[20]
1 

{20: array([0.1 , 0.14, 0.18, 0.22, 0.26, 0.3 , 0.34, 0.38, 0.42, 0.46, 0.5 ,
       0.54, 0.58, 0.62, 0.66, 0.7 , 0.74, 0.78, 0.82, 0.86, 0.9 ])}


In [34]:
# Path to configuration file.
config_path = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_config.yml"

output_df_list = []
for fault_component, fault_values in fault_value_bulk.items():
    for fault_value in fault_values:
        print(f'FAULT VALUE: {fault_value}')

        # Create instance of SWMM model.
        swmm = swmm_model.SWMM(config_path)

        # Configure model.
        swmm.configure_model()

        if fault_type == 'diameter':
            # Get initial diameter.
            initial_geometry = swmm.get_link_geometry(fault_component)
            initial_diameter = initial_geometry[0]
            
            # Handle percentage fault or absolute fault.
            if value_type == 'fraction':
                offset = initial_diameter * fault_value
            elif value_type == 'absolute':
                offset = fault_value
            else:
                raise ValueError('Value type incorrectly specified as', value_type)

            # Update the link diameter.
            update_geometry = initial_geometry.copy()
            update_geometry[0] = initial_diameter - offset
            swmm.set_link_geometry(fault_component, update_geometry)
            
            # Update the link offsets.
            swmm.set_link_offsets(fault_component, (offset, offset))

        elif fault_type == 'roughness':
            # Assign fault variables.
            link_id = fault_component
            n = fault_value
            
            # Get link roughness.
            link_n = swmm.get_link_roughness(link_id)
            
            # Handle percentage fault or absolute fault.
            if value_type == 'fraction':
                n = link_n * fault_value
            elif value_type == 'absolute':
                n = fault_value
            else:
                raise ValueError('Value type incorrectly specified as', value_type)

            # Update the link roughness.
            swmm.set_link_roughness(link_id, n)

        elif fault_type == 'seepage':
            # Assign fault variables.
            link_id = fault_component
            seepage_rate = fault_value

            # Update the link roughness.
            swmm.set_link_seepage(link_id, seepage_rate)

        elif fault_type == 'infiltration':
            # Assign fault variables.
            storage_id = fault_component
            Ksat = fault_value

            # Update the saturated hydraulic conductivity.
            swmm.set_storage_property(storage_id, 'Ksat', Ksat)

        elif fault_type == 'silting':
            # Assign fault variables.
            storage_id = fault_component
            silt_depth = fault_value

            # Get storage depth.
            Ds_init = swmm.get_storage_property(storage_id, 'MaxDepth')

            # Get conduit offsets.
            overflow_link_id = swmm.get_storage_outfall_link(storage_id)
            (in_offset_init, out_offset) = swmm.get_link_offsets(overflow_link_id)

            # Get the storage invert elevation.
            zi_init = swmm.get_storage_property(storage_id, 'Elev.')

            # Update the storage_depth.
            Ds = Ds_init - silt_depth
            swmm.set_storage_property(storage_id, 'MaxDepth', Ds)

            # Update conduit offset.
            in_offset = in_offset_init - silt_depth
            swmm.set_link_offsets(overflow_link_id, (in_offset, out_offset))

            # Update storage invert elevation.
            zi = zi_init + silt_depth
            swmm.set_storage_property(storage_id, 'Elev.', zi)

        # Run the simulation.
        swmm.run_simulation()

        # OUTPUT TO DATA FRAME
        # ---------------------------------------------------------------------
        # Get the node depths.
        node_depth_df = swmm.get_node_depth()

        # Get node flooding.
        node_flood_df = swmm.get_node_flooding()

        # Get node inflow.
        node_total_inflow_df = swmm.get_node_total_inflow()

        # Get link flow.
        link_flow_df = swmm.get_link_flow()

        # Get link depth.
        link_depth_df = swmm.get_link_depth()

        # Get link velocity.
        link_velocity_df = swmm.get_link_velocity()

        # Get precipitation time series.
        prcp_df = swmm.get_rainfall_timeseries()

        # Join data frames.
        fault_df = node_depth_df.merge(node_flood_df, on='datetime')
        fault_df = fault_df.merge(node_total_inflow_df, on='datetime')
        fault_df = fault_df.merge(link_flow_df, on='datetime')
        fault_df = fault_df.merge(link_depth_df, on='datetime')
        fault_df = fault_df.merge(link_velocity_df, on='datetime')
        fault_df = fault_df.merge(prcp_df, on='datetime')

        # Add fault type, component, and value.
        fault_df['fault_type'] = fault_type
        fault_df['fault_component'] = fault_component
        fault_df['fault_value'] = fault_value
        fault_df = fault_df.copy()

        output_df_list.append(fault_df)
        
        print('FAULT COMPONENTS', [df.loc[0, 'fault_component'] for df in output_df_list])
        print('SHAPE', fault_df.shape)

fault_df = pd.concat(output_df_list)

save_fp = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\Output\{}_impulse_fault_database.csv".format(fault_type)
print(fault_df.shape)
print(save_fp)
fault_df.to_csv(save_fp, index=False)

FAULT VALUE: 0.1
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.9 for 20 in 20
Set Storage 20 MaxDepth to 0.9
Updated InOffset value to -0.1 for 43 in 43
Updated OutOffset value to 0.0 for 43 in 43
Set Link 43 offsets to (-0.1, 0.0)
Updated Elev. value to 30.1 for 20 in 20
Set Storage 20 Elev. to 30.1
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:01:59 , 1
2020-01-01 00:02:49 , 1
2020-01-01 00:03:39 , 1
2020-01-01 00:04:29 , 1
2020-01-01 00:05:19 , 1
2020-01-01 00:06:09 , 2
2020-01-01 00:06:59 , 2
2020-01-01 00:07:49 , 2
2020-01-01 00:08:39 , 2
2020-01-01 00:09:29 , 3
2020-01-01 00:10:19 , 3
2020-01-01 00:11:09 , 3
2020-01-01 00:11:59 , 3
2020-01-01 00:12:49 , 4
2020-01-01 00:13:39 , 4
2020-01-01 00:14:29 , 4
2020-01-01 00:15:19 , 4
2020-01-01 00:16:09 , 4
2020-01-01 00:16:59 , 5
2020-01-01 00:17:49 , 5
2020-01-01 00:

2020-01-01 00:26:09 , 7
2020-01-01 00:26:59 , 7
2020-01-01 00:27:49 , 8
2020-01-01 00:28:39 , 8
2020-01-01 00:29:29 , 8
2020-01-01 00:30:19 , 8
2020-01-01 00:31:09 , 9
2020-01-01 00:31:59 , 9
2020-01-01 00:32:51 , 9
2020-01-01 00:33:46 , 9
2020-01-01 00:34:42 , 10
2020-01-01 00:35:42 , 10
2020-01-01 00:36:44 , 10
2020-01-01 00:37:48 , 10
2020-01-01 00:38:55 , 11
2020-01-01 00:40:05 , 11
2020-01-01 00:41:17 , 11
2020-01-01 00:42:32 , 12
2020-01-01 00:43:50 , 12
2020-01-01 00:45:10 , 13
2020-01-01 00:46:34 , 13
2020-01-01 00:48:01 , 13
2020-01-01 00:49:30 , 14
2020-01-01 00:51:03 , 14
2020-01-01 00:52:39 , 15
2020-01-01 00:54:18 , 15
2020-01-01 00:56:01 , 16
2020-01-01 00:57:47 , 16
2020-01-01 00:59:36 , 17
2020-01-01 01:01:28 , 17
2020-01-01 01:03:24 , 18
2020-01-01 01:05:24 , 18
2020-01-01 01:07:26 , 19
2020-01-01 01:09:33 , 19
2020-01-01 01:11:43 , 20
2020-01-01 01:13:56 , 21
2020-01-01 01:16:14 , 21
2020-01-01 01:18:35 , 22
2020-01-01 01:21:01 , 23
2020-01-01 01:23:30 , 23
2020-01-01

2020-01-01 03:30:12 , 58
2020-01-01 03:34:44 , 60
2020-01-01 03:39:18 , 61
2020-01-01 03:43:55 , 62
2020-01-01 03:48:34 , 63
2020-01-01 03:53:16 , 65
2020-01-01 03:57:59 , 66
2020-01-01 04:02:45 , 67
2020-01-01 04:07:34 , 69
2020-01-01 04:12:24 , 70
2020-01-01 04:17:16 , 71
2020-01-01 04:22:11 , 73
2020-01-01 04:27:08 , 74
2020-01-01 04:32:06 , 76
2020-01-01 04:37:07 , 77
2020-01-01 04:42:09 , 78
2020-01-01 04:47:13 , 80
2020-01-01 04:52:19 , 81
2020-01-01 04:57:27 , 83
2020-01-01 05:02:37 , 84
2020-01-01 05:07:48 , 86
2020-01-01 05:13:01 , 87
2020-01-01 05:18:16 , 88
2020-01-01 05:23:32 , 90
2020-01-01 05:28:50 , 91
2020-01-01 05:34:09 , 93
2020-01-01 05:39:30 , 94
2020-01-01 05:44:53 , 96
2020-01-01 05:50:17 , 97
2020-01-01 05:55:42 , 99
FAULT COMPONENTS [20, 20, 20, 20, 20]
SHAPE (360, 368)
FAULT VALUE: 0.30000000000000004
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.7 for 20 in 20
Set St

FAULT COMPONENTS [20, 20, 20, 20, 20, 20, 20]
SHAPE (360, 368)
FAULT VALUE: 0.38
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.62 for 20 in 20
Set Storage 20 MaxDepth to 0.62
Updated InOffset value to -0.38 for 43 in 43
Updated OutOffset value to 0.0 for 43 in 43
Set Link 43 offsets to (-0.38, 0.0)
Updated Elev. value to 30.38 for 20 in 20
Set Storage 20 Elev. to 30.38
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:01:59 , 1
2020-01-01 00:02:49 , 1
2020-01-01 00:03:39 , 1
2020-01-01 00:04:29 , 1
2020-01-01 00:05:19 , 1
2020-01-01 00:06:09 , 2
2020-01-01 00:06:59 , 2
2020-01-01 00:07:49 , 2
2020-01-01 00:08:39 , 2
2020-01-01 00:09:29 , 3
2020-01-01 00:10:19 , 3
2020-01-01 00:11:09 , 3
2020-01-01 00:11:59 , 3
2020-01-01 00:12:49 , 4
2020-01-01 00:13:39 , 4
2020-01-01 00:14:29 , 4
2020-01-01 00:15:19 , 4
2020-01-01 00:16

2020-01-01 00:26:59 , 7
2020-01-01 00:27:49 , 8
2020-01-01 00:28:39 , 8
2020-01-01 00:29:29 , 8
2020-01-01 00:30:19 , 8
2020-01-01 00:31:09 , 9
2020-01-01 00:31:59 , 9
2020-01-01 00:32:51 , 9
2020-01-01 00:33:46 , 9
2020-01-01 00:34:43 , 10
2020-01-01 00:35:43 , 10
2020-01-01 00:36:45 , 10
2020-01-01 00:37:49 , 11
2020-01-01 00:38:56 , 11
2020-01-01 00:40:06 , 11
2020-01-01 00:41:19 , 11
2020-01-01 00:42:35 , 12
2020-01-01 00:43:53 , 12
2020-01-01 00:45:14 , 13
2020-01-01 00:46:38 , 13
2020-01-01 00:48:05 , 13
2020-01-01 00:49:36 , 14
2020-01-01 00:51:09 , 14
2020-01-01 00:52:46 , 15
2020-01-01 00:54:26 , 15
2020-01-01 00:56:09 , 16
2020-01-01 00:57:55 , 16
2020-01-01 00:59:45 , 17
2020-01-01 01:01:39 , 17
2020-01-01 01:03:36 , 18
2020-01-01 01:05:36 , 18
2020-01-01 01:07:40 , 19
2020-01-01 01:09:47 , 19
2020-01-01 01:11:58 , 20
2020-01-01 01:14:13 , 21
2020-01-01 01:16:32 , 21
2020-01-01 01:18:54 , 22
2020-01-01 01:21:21 , 23
2020-01-01 01:23:53 , 23
2020-01-01 01:26:28 , 24
2020-01-0

2020-01-01 04:49:04 , 80
2020-01-01 04:54:12 , 82
2020-01-01 04:59:22 , 83
2020-01-01 05:04:33 , 85
2020-01-01 05:09:46 , 86
2020-01-01 05:15:01 , 88
2020-01-01 05:20:17 , 89
2020-01-01 05:25:35 , 90
2020-01-01 05:30:55 , 92
2020-01-01 05:36:16 , 93
2020-01-01 05:41:39 , 95
2020-01-01 05:47:03 , 96
2020-01-01 05:52:29 , 98
2020-01-01 05:57:56 , 99
FAULT COMPONENTS [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
SHAPE (360, 368)
FAULT VALUE: 0.58
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.42000000000000004 for 20 in 20
Set Storage 20 MaxDepth to 0.42000000000000004
Updated InOffset value to -0.58 for 43 in 43
Updated OutOffset value to 0.0 for 43 in 43
Set Link 43 offsets to (-0.58, 0.0)
Updated Elev. value to 30.58 for 20 in 20
Set Storage 20 Elev. to 30.58
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:01:59 , 1


2020-01-01 00:26:59 , 7
2020-01-01 00:27:49 , 8
2020-01-01 00:28:39 , 8
2020-01-01 00:29:29 , 8
2020-01-01 00:30:19 , 8
2020-01-01 00:31:09 , 9
2020-01-01 00:31:59 , 9
2020-01-01 00:32:52 , 9
2020-01-01 00:33:47 , 9
2020-01-01 00:34:44 , 10
2020-01-01 00:35:44 , 10
2020-01-01 00:36:47 , 10
2020-01-01 00:37:52 , 11
2020-01-01 00:39:00 , 11
2020-01-01 00:40:10 , 11
2020-01-01 00:41:23 , 11
2020-01-01 00:42:39 , 12
2020-01-01 00:43:58 , 12
2020-01-01 00:45:20 , 13
2020-01-01 00:46:45 , 13
2020-01-01 00:48:12 , 13
2020-01-01 00:49:43 , 14
2020-01-01 00:51:18 , 14
2020-01-01 00:52:55 , 15
2020-01-01 00:54:35 , 15
2020-01-01 00:56:19 , 16
2020-01-01 00:58:07 , 16
2020-01-01 00:59:58 , 17
2020-01-01 01:01:52 , 17
2020-01-01 01:03:50 , 18
2020-01-01 01:05:51 , 18
2020-01-01 01:07:56 , 19
2020-01-01 01:10:04 , 19
2020-01-01 01:12:16 , 20
2020-01-01 01:14:32 , 21
2020-01-01 01:16:52 , 21
2020-01-01 01:19:16 , 22
2020-01-01 01:21:44 , 23
2020-01-01 01:24:17 , 23
2020-01-01 01:26:54 , 24
2020-01-0

2020-01-01 04:07:07 , 69
2020-01-01 04:12:00 , 70
2020-01-01 04:16:54 , 71
2020-01-01 04:21:50 , 73
2020-01-01 04:26:48 , 74
2020-01-01 04:31:49 , 76
2020-01-01 04:36:51 , 77
2020-01-01 04:41:55 , 78
2020-01-01 04:47:02 , 80
2020-01-01 04:52:09 , 81
2020-01-01 04:57:19 , 83
2020-01-01 05:02:31 , 84
2020-01-01 05:07:44 , 85
2020-01-01 05:12:59 , 87
2020-01-01 05:18:15 , 88
2020-01-01 05:23:34 , 90
2020-01-01 05:28:53 , 91
2020-01-01 05:34:15 , 93
2020-01-01 05:39:38 , 94
2020-01-01 05:45:02 , 96
2020-01-01 05:50:28 , 97
2020-01-01 05:55:55 , 99
FAULT COMPONENTS [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
SHAPE (360, 368)
FAULT VALUE: 0.78
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.21999999999999997 for 20 in 20
Set Storage 20 MaxDepth to 0.21999999999999997
Updated InOffset value to -0.78 for 43 in 43
Updated OutOffset value to 0.0 for 43 in 43
Set Link 43 offsets t

FAULT COMPONENTS [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
SHAPE (360, 368)
FAULT VALUE: 0.86
Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00
Updated MaxDepth value to 0.14 for 20 in 20
Set Storage 20 MaxDepth to 0.14
Updated InOffset value to -0.86 for 43 in 43
Updated OutOffset value to 0.0 for 43 in 43
Set Link 43 offsets to (-0.86, 0.0)
Updated Elev. value to 30.86 for 20 in 20
Set Storage 20 Elev. to 30.86
INP PATH: C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_tmp.inp
2020-01-01 00:00:01 , 0
2020-01-01 00:01:59 , 1
2020-01-01 00:02:49 , 1
2020-01-01 00:03:39 , 1
2020-01-01 00:04:29 , 1
2020-01-01 00:05:19 , 1
2020-01-01 00:06:09 , 2
2020-01-01 00:06:59 , 2
2020-01-01 00:07:49 , 2
2020-01-01 00:08:39 , 2
2020-01-01 00:09:29 , 3
2020-01-01 00:10:19 , 3
2020-01-01 00:11:09 , 3
2020-01-01 00:11:59 , 3
2020-01-01 00:12:49 , 4
2020-01-01 00:13:39 , 4
2020-01-01 00:14

In [19]:
fault_df.shape

(3240, 368)

## Unused code

In [ ]:
# Changing diameter of a weir.
# Assign fault variables.
weir_id = fault_component
offset = fault_value

# Get initial weir height.
initial_geometry = swmm.get_weir_geometry(weir_id)
initial_height = initial_geometry[1]

# Handle percentage fault or absolute fault.
if value_type == 'percentage':
    offset = initial_height * offset
elif value_type == 'absolute':
    offset = offset
else:
    raise ValueError('Value type incorrectly specified as', value_type)

# Get initial weir crest height.
initial_crest_height = swmm.get_weir_property(weir_id, 'CrestHt')

# Update the weir height.
update_geometry = initial_geometry.copy()
update_geometry[1] = initial_height - offset
swmm.set_weir_geometry(weir_id, update_geometry)

# Update the weir crest height.
swmm.set_weir_property(weir_id, 'CrestHt', offset)